In [2]:
import requests
import json
import pandas as pd
from datetime import datetime as dt
import pytz
import time
TD_CONSUMER_KEY = 'GRWWHXWKKCNAZQYOTTGN6OZ7NBKXHTU1'
EST = pytz.timezone('US/Eastern')
DT_FORMAT = '%Y-%m-%d %H:%M:%S'
DT_FILE_FORMAT = '%Y%m%d%H%M%S'

pd.set_option('display.max_columns', None)

In [155]:
def get_stock_quotes(stock_tickers:list):
    endpoint = 'https://api.tdameritrade.com/v1/marketdata/quotes?'
    page = requests.get(url=endpoint,
                        params={'apikey' : TD_CONSUMER_KEY,
                                'symbol' : ",".join(stock_tickers)})
    content = json.loads(page.content)
    
    if content == {}:
        print("Error: Failed to get stock quotes of", stock_tickers)
        return 0
    return content

In [268]:
def convert_epoch_time_in_milliseconds_into_datetime_est(epoch_time : int):
    dt_result = dt.utcfromtimestamp(epoch_time / 1000) # epoch -> utc
    dt_result = dt_result.replace(tzinfo=pytz.UTC) # make aware of timezone UTC
    dt_result = dt_result.astimezone(EST).strftime(DT_FORMAT) # utc -> est
    return dt_result
    
def preprocess_stock_quotes(json_data):
    # convert json_data into pandas dataframe
    df = pd.DataFrame(json_data).T
    df.index.name = 'ticker'
    
    # convert time columns into %Y-%m-%d %H:%M:%S EST
    time_cols = ['quoteTimeInLong', 'tradeTimeInLong', 'regularMarketTradeTimeInLong']
    time_cols_new = ['quoteTime', 'tradeTime', 'timestamp']
    replace_dict = dict(zip(time_cols, time_cols_new))
    df.loc[:, time_cols] = df[time_cols].applymap(convert_epoch_time_in_milliseconds_into_datetime_est)
    df.rename(columns=replace_dict, inplace=True)
    
    # set timestamp as index
    df = df.reset_index().set_index('timestamp')
    
    return df

In [158]:
ticker_list = ['SPY', 'TSLA', 'QQQ', 'SPX', 'AMZN', 'AAPL','AMD','IWM','VIX','NVDA','TQQQ','META','BABA','AMC','HYG','MSFT','GOOGL','DIS','UVXY','NFLX','C','COIN','GOOG','TLT','NIO','XLE','SQQQ','FXI','EWZ','SOXL',
              'ARKK','SLV','EEM','SOFI','XLF','SHOP','MARA','CCL','DIA','PYPL','XOM','PBR','BAC','WMT','UBER',
              'DKNG','F','AAL','INTC','GLD','PLTR','RIVN','WBD','KO','LQD','GM']


,ticker,assetType,assetMainType,cusip,assetSubType,symbol,description,bidPrice,bidSize,bidId,askPrice,askSize,askId,lastPrice,lastSize,lastId,openPrice,highPrice,lowPrice,bidTick,closePrice,netChange,totalVolume,quoteTime,tradeTime,mark,exchange,exchangeName,marginable,shortable,volatility,digits,52WkHigh,52WkLow,nAV,peRatio,divAmount,divYield,divDate,securityStatus,regularMarketLastPrice,regularMarketLastSize,regularMarketNetChange,netPercentChangeInDouble,markChangeInDouble,markPercentChangeInDouble,regularMarketPercentChangeInDouble,delayed,realtimeEntitled
timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2022-11-23 20:00:00,SPY,ETF,EQUITY,78462F103,ETF,SPY,SPDR S&P 500,403.07,200,P,403.11,300,P,402.42,934700,P,399.55,402.93,399.31,,402.42,0.0,68261628,2022-11-23 19:59:59,2022-11-23 20:00:00,402.42,p,PACIFIC,True,True,0.0049,2,479.98,348.11,0.0,0.0,6.1757,1.53,2022-09-16 00:00:00.000,Normal,402.42,1,0.0,0.0,0.0,0.0,0.0,True,False
2022-11-23 16:00:00,TSLA,EQUITY,EQUITY,88160R101,,TSLA,"Tesla, Inc. - Common Stock",184.75,600,P,184.81,4300,P,184.81,0,P,173.57,183.62,172.5,,183.2,1.61,109536709,2022-11-23 19:59:59,2022-11-23 19:59:59,183.2,q,NASD,True,True,0.1147,4,402.6667,166.185,0.0,46.98,0.0,0.0,,Normal,183.2,16994,0.0,0.8788,0.0,0.0,0.0,True,False
2022-11-23 16:00:00,QQQ,ETF,EQUITY,46090E103,ETF,QQQ,"Invesco QQQ Trust, Series 1",289.53,300,P,289.55,1500,Z,289.55,0,K,285.97,289.46,285.79,,288.82,0.73,46389967,2022-11-23 19:59:56,2022-11-23 19:59:59,288.82,q,NASD,True,True,0.0089,4,404.58,254.26,0.0,0.0,1.9711,0.68,2022-09-19 00:00:00.000,Normal,288.82,3752,0.0,0.2528,0.0,0.0,0.0,True,False
2022-11-23 16:00:00,AMZN,EQUITY,EQUITY,023135106,,AMZN,"Amazon.com, Inc. - Common Stock",94.6,4600,P,94.62,2200,P,94.62,0,D,93.24,94.58,92.83,,94.13,0.49,59414684,2022-11-23 19:59:57,2022-11-23 19:59:59,94.13,q,NASD,True,True,0.0142,4,181.675,85.87,0.0,82.62,0.0,0.0,,Normal,94.13,36535,0.0,0.5206,0.0,0.0,0.0,True,False
2022-11-23 16:00:00,AAPL,EQUITY,EQUITY,037833100,,AAPL,Apple Inc. - Common Stock,151.11,500,P,151.15,400,P,151.145,0,D,149.45,151.83,149.34,,151.07,0.075,58301395,2022-11-23 19:59:59,2022-11-23 19:59:59,151.07,q,NASD,True,True,0.0091,4,182.94,129.04,0.0,24.6112,0.92,0.61,2022-11-04 00:00:00.000,Normal,151.07,40030,0.0,0.0496,0.0,0.0,0.0,True,False
2022-11-23 16:00:01,AMD,EQUITY,EQUITY,007903107,,AMD,"Advanced Micro Devices, Inc. - Common Stock",76.63,100,P,76.74,100,P,76.64,1000,P,75.49,78.22,75.3,,76.4,0.24,81623628,2022-11-23 19:59:47,2022-11-23 19:59:29,76.4,q,NASD,True,True,0.0104,4,164.4599,54.57,0.0,45.61,0.0,0.0,,Normal,76.4,17447,0.0,0.3141,0.0,0.0,0.0,True,False
2022-11-23 20:00:00,IWM,ETF,EQUITY,464287655,ETF,IWM,iShares Russell 2000 ETF,185.62,700,P,185.65,300,P,185.19,292500,P,184.29,185.78,183.55,,185.19,0.0,19758783,2022-11-23 20:00:00,2022-11-23 20:00:00,185.19,p,PACIFIC,True,True,0.0073,2,233.0,162.5,0.0,0.0,2.4207,1.31,2022-09-26 00:00:00.000,Normal,185.19,2,0.0,0.0,0.0,0.0,0.0,True,False
2022-11-23 16:00:00,NVDA,EQUITY,EQUITY,67066G104,,NVDA,NVIDIA Corporation - Common Stock,165.8,1600,P,165.89,1500,P,165.845,0,D,160.98,165.27,160.4801,,165.19,0.655,42724093,2022-11-23 19:59:54,2022-11-23 19:59:55,165.19,q,NASD,True,True,0.0164,4,334.12,108.13,0.0,68.2172,0.16,0.1,2022-11-30 00:00:00.000,Normal,165.19,13202,0.0,0.3965,0.0,0.0,0.0,True,False
2022-11-23 16:00:00,TQQQ,ETF,EQUITY,74347X831,ETF,TQQQ,ProShares UltraPro QQQ,22.88,1400,Q,22.89,41200,P,22.89,100,D,22.05,22.87,22.01,,22.73,0.16,178783485,2022-11-23 19:59:59,2022-11-23 19:59:59,22.73,q,NASD,True,True,0.0251,4,87.49,16.32,0.0,0.0,0.0,0.0,,Normal,22.73,3193,0.0,0.7039,0.0,0.0,0.0,True,False


In [163]:
def get_option_quotes(stock_ticker:str):
    endpoint = 'https://api.tdameritrade.com/v1/marketdata/chains?'
    page = requests.get(url=endpoint,
                        params={'apikey' : TD_CONSUMER_KEY,
                                'symbol' : stock_ticker,
                                'includeQuotes' : True
                               })
    content = json.loads(page.content)
    
    if content == {}:
        print("Error: Failed to get stock quotes of", stock_tickers)
        return 0
    if content['status'] != 'SUCCESS':
        print("Error: Request status is not SUCCESS. Instead, it is", content['status'])
    return content

In [269]:
def json_normalize_option_chain(option_chain):
    df = pd.DataFrame(option_chain).T # each row is a maturity, each column is a strike
    df.reset_index(inplace=True)
    df = df.melt(id_vars=['index']) # each row is a combo of maturity and strike, its value is a list of size 1, the element is a dictionary
    df.dropna(inplace=True) 
    
    df_sub = df['index'].str.split(':', expand=True) # index columns is e.g. 2022-12-31:10 (ExpiryDate:DaysToExpiry)
    df_sub.columns = ['T', 'daysToExpiry']
    
    res = df.value.apply(lambda elem: pd.Series(elem[0])) # convert each cell into a pandas series
    cols = res.columns
    
    res['T'] = df_sub['T']
    res['daysToExpiry'] = df_sub['daysToExpiry']
    res['K'] = df['variable']
    res = res[['T','K']+list(cols)+['daysToExpiry']] # make T, K the first two columns
    
    res.reset_index(drop=True, inplace=True)
    res.set_index('symbol', inplace=True)
    return res

def preprocess_option_chains(json_data):
    volatility, num_contracts = json_data['volatility'], json_data['numberOfContracts']
    call_option_chain = json_normalize_option_chain(json_data['callExpDateMap'])
    put_option_chain = json_normalize_option_chain(json_data['putExpDateMap'])
    all_option_chain = pd.concat([call_option_chain, put_option_chain])
    
    # convert time columns into %Y-%m-%d %H:%M:%S EST
    time_cols = ['quoteTimeInLong', 'tradeTimeInLong', 'lastTradingDay']
    time_cols_new = ['quoteTime', 'tradeTime', 'lastTradingDay']
    replace_dict = dict(zip(time_cols, time_cols_new))
    all_option_chain.loc[:, time_cols] = all_option_chain[time_cols].applymap(convert_epoch_time_in_milliseconds_into_datetime_est)
    all_option_chain.rename(columns=replace_dict, inplace=True)
    return volatility, num_contracts, all_option_chain

In [289]:
current_timestamp = dt.today().strftime(DT_FILE_FORMAT)
meta_info_list = []

ticker_list = ['SPY', 'TSLA', 'QQQ', 'SPX', 'AMZN', 'AAPL','AMD','IWM','VIX','NVDA','TQQQ','META','BABA','AMC','HYG','MSFT','GOOGL','DIS','UVXY','NFLX','C','COIN','GOOG','TLT','NIO','XLE','SQQQ','FXI','EWZ','SOXL',
              'ARKK','SLV','EEM','SOFI','XLF','SHOP','MARA','CCL','DIA','PYPL','XOM','PBR','BAC','WMT','UBER',
              'DKNG','F','AAL','INTC','GLD','PLTR','RIVN','WBD','KO','LQD','GM']

res = get_stock_quotes(ticker_list)
df = preprocess_stock_quotes(res)
df.to_csv("stock_{}.csv".format(current_timestamp))
meta_info_list.append({'stock_list' : df['symbol'].to_list()})

for option_ticker in ticker_list:
    try:
        print("Loading {} option chain".format(option_ticker))
        res = get_option_quotes(option_ticker)
        vol, num, df = preprocess_option_chains(res)
        df.to_csv("option_{}_{}.csv".format(option_ticker, current_timestamp))
        meta_info_list.append({'option_{}'.format(option_ticker) : [option_ticker, vol, num]})
    except:
        print("Error when loading {} option chain".format(option_ticker))

In [5]:
dt.today().astimezone(pytz.UTC)

datetime.datetime(2022, 11, 25, 22, 23, 57, 876656, tzinfo=<UTC>)

In [293]:
vol

29.0

In [294]:
num

2066

In [295]:
df

,T,K,putCall,description,exchangeName,bid,ask,last,mark,bidSize,askSize,bidAskSize,lastSize,highPrice,lowPrice,openPrice,closePrice,totalVolume,tradeDate,tradeTime,quoteTime,netChange,volatility,delta,gamma,theta,vega,rho,openInterest,timeValue,theoreticalOptionValue,theoreticalVolatility,optionDeliverablesList,strikePrice,expirationDate,daysToExpiration,expirationType,lastTradingDay,multiplier,settlementType,deliverableNote,isIndexOption,percentChange,markChange,markPercentChange,intrinsicValue,mini,inTheMoney,nonStandard,pennyPilot,daysToExpiry
symbol,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
AAPL_112522C70,2022-11-25,70.0,CALL,AAPL Nov 25 2022 70 Call (Weekly),OPR,77.95,78.15,80.30,78.05,34,11,34X11,0,0.0,0.0,0.0,81.08,0,None,2022-11-23 13:41:05,2022-11-25 11:23:02,-0.78,549.246,1.0,0.0,0.006,0.000,0.0,7,2.25,78.056,29.0,None,70.0,1669410000000,0,S,2022-11-25 20:00:00,100.0,,,None,-0.96,-3.03,-3.74,78.05,False,True,False,True,0
AAPL_120222C70,2022-12-02,70.0,CALL,AAPL Dec 2 2022 70 Call (Weekly),OPR,77.95,78.25,0.00,78.10,32,6,32X6,0,0.0,0.0,0.0,81.12,0,None,1969-12-31 19:00:00,2022-11-25 11:22:52,0.00,163.264,1.0,0.0,-0.017,0.000,0.015,0,0.05,78.163,29.0,None,70.0,1670014800000,7,S,2022-12-02 20:00:00,100.0,,,None,0.00,-3.02,-3.73,78.05,False,True,False,True,7
AAPL_120922C70,2022-12-09,70.0,CALL,AAPL Dec 9 2022 70 Call (Weekly),OPR,77.95,78.30,71.90,78.13,48,44,48X44,0,0.0,0.0,0.0,81.18,0,None,2022-11-10 09:30:58,2022-11-25 11:22:48,-9.28,NaN,NaN,NaN,NaN,0.001,NaN,1,-6.15,NaN,29.0,None,70.0,1670619600000,14,S,2022-12-09 20:00:00,100.0,,,None,-11.43,-3.05,-3.76,78.05,False,True,False,True,14
AAPL_121622C70,2022-12-16,70.0,CALL,AAPL Dec 16 2022 70 Call,OPR,78.10,78.35,81.40,78.22,6,38,6X38,0,0.0,0.0,0.0,81.24,0,None,2022-11-23 10:00:02,2022-11-25 11:22:59,0.16,97.348,1.002,0.0,-0.017,0.001,0.041,388,3.35,78.4,29.0,None,70.0,1671224400000,21,R,2022-12-16 20:00:00,100.0,,,None,0.20,-3.01,-3.71,78.05,False,True,False,True,21
AAPL_122322C70,2022-12-23,70.0,CALL,AAPL Dec 23 2022 70 Call (Weekly),OPR,78.10,78.45,0.00,78.28,51,7,51X7,0,0.0,0.0,0.0,81.36,0,None,1969-12-31 19:00:00,2022-11-25 11:22:48,0.00,84.721,1.003,0.0,-0.017,0.001,0.055,0,0.23,78.523,29.0,None,70.0,1671829200000,28,S,2022-12-23 20:00:00,100.0,,,None,0.00,-3.08,-3.79,78.05,False,True,False,True,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AAPL_011924P310,2024-01-19,310.0,PUT,AAPL Jan 19 2024 310 Put,OPR,161.00,162.50,168.86,161.75,82,45,82X45,0,0.0,0.0,0.0,158.93,0,None,2022-10-13 14:34:23,2022-11-25 11:23:00,9.93,29.108,-1.0,0.0,0.0,0.000,0.0,1,6.91,161.944,29.0,None,310.0,1705698000000,420,R,2024-01-19 20:00:00,100.0,,,None,6.25,2.82,1.77,161.95,False,True,False,True,420
AAPL_062124P310,2024-06-21,310.0,PUT,AAPL Jun 21 2024 310 Put,OPR,160.45,163.00,168.53,161.73,140,46,140X46,0,0.0,0.0,0.0,158.93,0,None,2022-10-07 11:12:54,2022-11-25 11:22:36,9.60,28.495,-1.0,0.0,0.0,0.000,0.0,0,6.58,161.944,29.0,None,310.0,1719000000000,574,R,2024-06-21 20:00:00,100.0,,,None,6.04,2.80,1.76,161.95,False,True,False,True,574
AAPL_011725P310,2025-01-17,310.0,PUT,AAPL Jan 17 2025 310 Put,OPR,159.85,164.35,161.33,162.10,50,50,50X50,0,0.0,0.0,0.0,158.93,0,None,2022-11-22 11:04:52,2022-11-25 11:22:15,2.40,28.014,-1.0,0.0,0.0,0.000,0.0,0,-0.62,161.944,29.0,None,310.0,1737147600000,784,R,2025-01-17 20:00:00,100.0,,,None,1.51,3.17,1.99,161.95,False,True,False,True,784


In [258]:
res['volatility']

29.0

In [172]:
res['numberOfContracts']

2066

In [254]:
tmp1 = json_normalize_option_chain(res['callExpDateMap'])

In [255]:
tmp2 = json_normalize_option_chain(res['putExpDateMap'])

In [259]:
pd.concat([tmp1, tmp2])

,T,K,putCall,description,exchangeName,bid,ask,last,mark,bidSize,askSize,bidAskSize,lastSize,highPrice,lowPrice,openPrice,closePrice,totalVolume,tradeDate,tradeTimeInLong,quoteTimeInLong,netChange,volatility,delta,gamma,theta,vega,rho,openInterest,timeValue,theoreticalOptionValue,theoreticalVolatility,optionDeliverablesList,strikePrice,expirationDate,daysToExpiration,expirationType,lastTradingDay,multiplier,settlementType,deliverableNote,isIndexOption,percentChange,markChange,markPercentChange,intrinsicValue,inTheMoney,mini,pennyPilot,nonStandard,daysToExpiry
symbol,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
AAPL_112522C70,2022-11-25,70.0,CALL,AAPL Nov 25 2022 70 Call (Weekly),OPR,78.80,82.00,80.30,80.40,3,30,3X30,0,81.0,80.3,0.0,81.08,2,None,1669228865560,1669237195129,0.08,357.453,1.0,0.0,-0.034,0.000,0.003,10,-0.77,81.1,29.0,None,70.0,1669410000000,1,S,1669424400000,100.0,,,None,0.10,-0.68,-0.84,81.07,True,False,True,False,1
AAPL_120222C70,2022-12-02,70.0,CALL,AAPL Dec 2 2022 70 Call (Weekly),OPR,79.00,82.05,0.00,80.53,56,56,56X56,0,0.0,0.0,0.0,81.12,0,None,0,1669237156840,0.00,157.269,1.0,0.0,-0.018,0.000,0.017,0,-0.54,81.198,29.0,None,70.0,1670014800000,8,S,1670029200000,100.0,,,None,0.00,-0.60,-0.74,81.07,True,False,True,False,8
AAPL_120922C70,2022-12-09,70.0,CALL,AAPL Dec 9 2022 70 Call (Weekly),OPR,79.40,82.05,71.90,80.72,57,56,57X56,0,0.0,0.0,0.0,81.18,0,None,1668090658817,1669237185557,-8.46,139.565,0.999,0.0,-0.024,0.002,0.03,1,-9.17,81.316,29.0,None,70.0,1670619600000,15,S,1670634000000,100.0,,,None,-10.42,-0.45,-0.56,81.07,True,False,True,False,15
AAPL_121622C70,2022-12-16,70.0,CALL,AAPL Dec 16 2022 70 Call,OPR,79.00,82.00,81.40,80.50,61,56,61X56,0,81.4,81.4,0.0,81.24,2,None,1669215602659,1669237187541,0.98,119.278,0.999,0.0,-0.023,0.003,0.043,386,0.33,81.44,29.0,None,70.0,1671224400000,22,R,1671238800000,100.0,,,None,1.20,-0.74,-0.91,81.07,True,False,True,False,22
AAPL_122322C70,2022-12-23,70.0,CALL,AAPL Dec 23 2022 70 Call (Weekly),OPR,80.80,82.25,0.00,81.53,50,51,50X51,0,0.0,0.0,0.0,81.36,0,None,0,1669237186058,0.00,104.475,1.001,0.0,-0.027,0.004,0.056,0,0.46,81.713,29.0,None,70.0,1671829200000,29,S,1671843600000,100.0,,,None,0.00,0.17,0.20,81.07,True,False,True,False,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AAPL_011924P310,2024-01-19,310.0,PUT,AAPL Jan 19 2024 310 Put,OPR,156.85,160.20,168.86,158.52,50,50,50X50,0,0.0,0.0,0.0,158.93,0,None,1665686063512,1669237191936,9.04,29.079,-1.0,0.0,0.0,0.000,0.0,1,9.93,158.93,29.0,None,310.0,1705698000000,421,R,1705712400000,100.0,,,None,5.69,-0.40,-0.25,158.93,True,False,True,False,421
AAPL_062124P310,2024-06-21,310.0,PUT,AAPL Jun 21 2024 310 Put,OPR,157.30,160.80,168.53,159.05,50,50,50X50,0,0.0,0.0,0.0,158.93,0,None,1665155574812,1669237192276,8.71,27.721,-1.0,0.0,0.0,0.000,0.0,0,9.60,158.93,29.0,None,310.0,1719000000000,575,R,1719014400000,100.0,,,None,5.48,0.12,0.08,158.93,True,False,True,False,575
AAPL_011725P310,2025-01-17,310.0,PUT,AAPL Jan 17 2025 310 Put,OPR,156.50,161.10,161.33,158.80,52,50,52X50,0,0.0,0.0,0.0,158.93,0,None,1669133092407,1669237176490,1.51,27.323,-1.0,0.0,0.0,0.000,0.0,0,2.40,158.93,29.0,None,310.0,1737147600000,785,R,1737162000000,100.0,,,None,0.95,-0.13,-0.08,158.93,True,False,True,False,785
